In [ ]:
# Scraping tweets using snscrape package
# Date: 20.12.22
# Author: Laura Starzomska

In [5]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import regex as re
from operator import itemgetter

In [16]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from operator import itemgetter

# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list
# We append tweets with #COP27 minimum likes 300 in english language and posted between 30.10 and 7.11.22
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#COP27 min_faves:300 lang:en since:2022-10-31 until:2022-11-08').get_items()):
    if i>1000:
        break
    attributes_container.append([tweet.user.username, tweet.date, tweet.likeCount, tweet.retweetCount, tweet.sourceLabel, tweet.content])
    attributes_container = sorted(attributes_container, key=itemgetter(2), reverse = True)
# Creating a dataframe to load the list
tweets_df = pd.DataFrame(attributes_container, columns=["User", "Date Created", "Number of Likes", "Retweets", "Source of Tweet", "Tweet"])

/var/folders/f8/45gtfsk111s8zwsfwv07l26c0000gq/T/ipykernel_4376/3193908534.py:12: FutureWarning: content is deprecated, use rawContent instead
  attributes_container.append([tweet.user.username, tweet.date, tweet.likeCount, tweet.retweetCount, tweet.sourceLabel, tweet.content])


In [17]:
tweets_df

,User,Date Created,Number of Likes,Retweets,Source of Tweet,Tweet
0,DiEM_25,2022-11-07 11:13:33+00:00,89865,31482,Twitter Web App,The world's elite have arrived at #COP27 on hu...
1,RishiSunak,2022-11-07 20:30:43+00:00,52716,3196,Twitter Web App,Friends 🤝\nPartners 🌍\nAllies 🇬🇧🇫🇷\n\nGreat to...
2,_david_ho_,2022-11-07 12:32:10+00:00,28773,5980,Twitter for iPhone,"At this point, maybe we should ask women for h..."
3,JamesMelville,2022-11-06 23:25:23+00:00,22536,5666,Twitter for iPhone,“Climate change will ravage our planet without...
4,antonioguterres,2022-11-07 11:04:12+00:00,21342,8284,Twitter for iPhone,I have just warned global leaders at #COP27:\n...
...,...,...,...,...,...,...
292,collinrees,2022-11-07 09:13:34+00:00,393,175,Twitter for iPhone,“Decarbonisation means a phase-out of fossil f...
293,AlokSharma_RDG,2022-11-02 10:15:41+00:00,379,69,Twitter for iPhone,Delighted that @RishiSunak is attending #COP27...
294,harjeet11,2022-11-06 10:51:16+00:00,368,193,Twitter for Android,#LossAndDamage Finance agenda for #COP27 adopt...
295,katehamptongray,2022-10-31 15:13:39+00:00,347,135,Twitter for Android,Friends of COP advised #UK and @AlokSharma_RDG...


In [19]:
tweets_df['Tweet'] = tweets_df['Tweet'].replace(to_replace=r'http[^\s]+',value='',regex=True) # Get rid of links
tweets_df_2 = tweets_df
tweets_df_2 = tweets_df_2[tweets_df_2["Tweet"].str.len() >= 15] # We keep only tweets longer than 15 characters

def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet) # remove nicknames
    tweet = re.sub('http[^\s]+','',tweet) # remove links
    return tweet

tweets_df_2['Tweet'] = tweets_df_2['Tweet'].apply(remove_usernames_links) # Delete nicknames and links again
tweets_df_2['Tweet'] = tweets_df_2['Tweet'].str.strip()
tweets_df_2 = tweets_df_2[tweets_df_2.Tweet.str.len() >= 15]
tweets_df_2['Tweet'] = tweets_df_2['Tweet'].str.replace("\n", "") # Replace enter space
tweets_df_2['Date Created'] = tweets_df_2['Date Created'].apply(lambda a: pd.to_datetime(a).date()) # Transform date format
tweets_df_2

,User,Date Created,Number of Likes,Retweets,Source of Tweet,Tweet
0,DiEM_25,2022-11-07,89865,31482,Twitter Web App,The world's elite have arrived at #COP27 on hu...
1,RishiSunak,2022-11-07,52716,3196,Twitter Web App,Friends 🤝Partners 🌍Allies 🇬🇧🇫🇷Great to meet wi...
2,_david_ho_,2022-11-07,28773,5980,Twitter for iPhone,"At this point, maybe we should ask women for h..."
3,JamesMelville,2022-11-06,22536,5666,Twitter for iPhone,“Climate change will ravage our planet without...
4,antonioguterres,2022-11-07,21342,8284,Twitter for iPhone,I have just warned global leaders at #COP27:We...
...,...,...,...,...,...,...
292,collinrees,2022-11-07,393,175,Twitter for iPhone,“Decarbonisation means a phase-out of fossil f...
293,AlokSharma_RDG,2022-11-02,379,69,Twitter for iPhone,Delighted that is attending #COP27 - complete...
294,harjeet11,2022-11-06,368,193,Twitter for Android,#LossAndDamage Finance agenda for #COP27 adopt...
295,katehamptongray,2022-10-31,347,135,Twitter for Android,"Friends of COP advised #UK and on #COP26, hav..."


In [78]:
tweets_df_2.to_csv('text-query-tweets.csv', sep=',', index=False)
tweets_df_2.to_excel('text-query-tweets.xlsx')
